In [44]:
from nltk import PCFG,ViterbiParser,InsideChartParser

In [45]:
grammar=PCFG.fromstring("""
S -> NN VP  [0.50] | VP NP  [0.50]
NP -> NN PP [0.70] | PP NN   [0.30]
VP -> VB NP [0.30] | VP NP   [0.20] | VB NN [0.35] | VB PP   [0.15] 
PP ->  P VP  [0.50] | P NN   [0.50]
NN -> 'students'   [0.10] | 'football'   [0.15] | 'friends'  [0.20] |'children'  [0.15] | 'cricket'   [0.15] | 'painting'   [0.25] 
VB -> 'play'  [0.20] | 'enjoy'   [0.35] | 'watch'   [0.20] | 'like'   [0.15] | 'listen'   [0.10] 
P -> 'with'  [0.70] | 'without'   [0.30]
""")

In [46]:
parser1=ViterbiParser(grammar)
parser2=InsideChartParser(grammar)

In [47]:
sentence="students play football with friends"
tokens=sentence.split()

In [48]:
tokens

['students', 'play', 'football', 'with', 'friends']

In [49]:
for tree in parser1.parse(tokens):
    print(tree)

(S
  (NN students)
  (VP
    (VB play)
    (NP (NN football) (PP (P with) (NN friends))))) (p=2.205e-05)


In [50]:
for tree in parser2.parse(tokens):
    print(tree)

(S
  (NN students)
  (VP
    (VB play)
    (NP (NN football) (PP (P with) (NN friends))))) (p=2.205e-05)


In [51]:
import numpy as np
def cyk_algo(grammar,sentence):
    n=len(sentence)
    table=np.empty((n,n),dtype=object)
    for i,word in enumerate(sentence):
        table[i,i]=[(p.lhs(),p.prob()) for p in grammar.productions() if len(p.rhs())==1 and p.rhs()[0]==word]
    for span in range(2,n+1):
        for  i in range(n-span+1):
            table[i,i+span-1]=[(p.lhs(),lp*rp*p.prob()) for k in range(i,i+span-1)
                               for l,lp in (table[i,k] or []) for r,rp in (table[k+1,i+span-1] or [])                               
                               for p in grammar.productions() if len(p.rhs())==2 and (l,r)==p.rhs()]
    return table
        

In [55]:
def check_sentence(sentence,grammar):
    words,result=sentence.split(),cyk_algo(grammar,sentence.split()) [0,len(sentence.split())-1] or []
    prob=max((p for nt,p in result if str(nt)=="S"),default=None)
    print(f"garmmar correct:{prob}" if prob else "grammar error")
    return prob

grammar=PCFG.fromstring("""
S -> NN VP  [0.50] | VP NP  [0.50]
NP -> NN PP [0.70] | PP NN   [0.30]
VP -> VB NP [0.30] | VP NP   [0.20] | VB NN [0.35] | VB PP   [0.15] 
PP ->  P VP  [0.50] | P NN   [0.50]
NN -> 'students'   [0.10] | 'football'   [0.15] | 'friends'  [0.20] |'children'  [0.15] | 'cricket'   [0.15] | 'painting'   [0.25] 
VB -> 'play'  [0.20] | 'enjoy'   [0.35] | 'watch'   [0.20] | 'like'   [0.15] | 'listen'   [0.10] 
P -> 'with'  [0.70] | 'without'   [0.30]
""")
if grammar:
    final_prob=check_sentence("students play football with friends",grammar)

garmmar correct:2.205e-05
